In [1]:
from helperfunctions import train_test_split
import pandas as pd
from collections import Counter
import numpy as np

In [460]:
df = pd.read_csv("Iris.csv")
df.head()
df.drop(columns='Id', inplace=True)

In [461]:
df.shape

(150, 5)

In [483]:
df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
0,5.1,3.5,1.4,0.2,Iris-setosa
1,4.9,3.0,1.4,0.2,Iris-setosa
2,4.7,3.2,1.3,0.2,Iris-setosa
3,4.6,3.1,1.5,0.2,Iris-setosa
4,5.0,3.6,1.4,0.2,Iris-setosa


In [581]:
train_df, test_df = train_test_split(df)
train_df.shape, test_df.shape

((123, 5), (30, 5))

In [582]:
test_df.head()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
82,5.8,2.7,3.9,1.2,Iris-versicolor
144,6.7,3.3,5.7,2.5,Iris-virginica
47,4.6,3.2,1.4,0.2,Iris-setosa
18,5.7,3.8,1.7,0.3,Iris-setosa
39,5.1,3.4,1.5,0.2,Iris-setosa


In [583]:
dtc = DecisionTreeClassifier(min_samples=2, max_depth=3)

In [584]:
dtc

In [585]:
dtc.fit(train_df)

In [586]:
dtc.predict(test_df)

('Accuracy:', 1.0)

In [588]:
class DecisionTreeClassifier:
    '''Can handle numeric features and dataframes that are free of missing data.
    If missing values, please impute first.'''

    def __init__(self, min_samples=2, max_depth=5, tree=None):
        
        self.max_depth = max_depth
        self.min_samples = min_samples
        self.tree = None
        
    def fit(self, train_df):
        
        # run method decision_tree and set it as the self.tree attribute
        self.tree = self.decision_tree(train_df, self.min_samples, self.max_depth)
        
        return self

    def entropy(self, data):

        # labels for input data
        labels = data[:,-1]

        # get unique labels along with their counts
        _, label_counts = np.unique(labels, return_counts=True)

        # array of probabilities of each label
        probabilities = label_counts / label_counts.sum()
        
        # return entropy
        return sum(probabilities * -np.log2(probabilities))

    def make_classification(self, data):

        '''Once the max depth or min samples or purity is 1, 
        we classify the data with whatever the majority of the labels are'''

        # labels for input data
        labels = data[:,-1]

        # instantiate a Counter object on the labels
        counter = Counter(labels)

        # return the most common class/label
        return counter.most_common(1)[0][0]

    def split_data(self, data, split_feature, split_threshold):

        # array of only the split_feature
        feature_values = data[:,split_feature]

        # array where feature values do not exceed threshold, array where does exceed threshold
        return data[feature_values <= split_threshold], data[feature_values > split_threshold]

    def overall_entropy(self, data_below_threshold, data_above_threshold):

        '''Overall entropy'''

        p_below = len(data_below_threshold) / (len(data_below_threshold) + len(data_above_threshold))
        p_above = len(data_above_threshold) / (len(data_below_threshold) + len(data_above_threshold))

        return (p_below * self.entropy(data_below_threshold)) + (p_above * self.entropy(data_above_threshold))

    def potential_splits(self, data):
        
        # dictionary of splits
        potential_splits = {}

        # store the number of features (not including labels/target)
        n_features = len(data[0])

        # for each feature in possible features
        for feature in range(n_features - 1):

            # for our dictionary, each feature should be a key
            potential_splits[feature] = []

            # we need to iterate through each feature's unique values
            unique_values_for_feature = np.unique(data[:, feature])

            for index in range(len(unique_values_for_feature)):
                
                if index != 0:

                    # we need to partition the data, we need the midpoint between the unique values
                    current = unique_values_for_feature[index]
                    prev = unique_values_for_feature[index - 1]
                    midpoint = (current + prev) / 2

                    # for our dictionary each value should be a midpoint between the 
                    # unique values for that feature
                    potential_splits[feature].append(midpoint)

        # return dictionary
        return potential_splits

    def find_best_split(self, data, potential_splits):

        lowest_entropy = 9999

        # for each dictionary key
        for key in potential_splits:
            
            # for each value for that key
            for value in potential_splits[key]:

                # split our data into on that threshold (value)
                data_below_threshold, data_above_threshold = self.split_data(
                    data=data, 
                    split_feature=key,
                    split_threshold=value)
                
                # calculate entropy at this split
                entropy_for_this_split = self.overall_entropy(data_below_threshold, data_above_threshold)

                # if entropy at this split is lower than the lowest entropy found so far
                if entropy_for_this_split < lowest_entropy:

                    # the entropy at this split is now the lowest 
                    lowest_entropy = entropy_for_this_split

                    # keep a record of this key, value pair
                    best_split_feature = key
                    best_split_threshold = value

        # return the best potential split
        return best_split_feature, best_split_threshold
    

    def purity(self, data):

        # last column of the df must be the labels!
        labels = data[:, -1]

        # if data has only one kind of label
        if len(np.unique(labels)) == 1:

            # it is pure
            return True

        # if data has a few different labels still
        else:

            # it isn't pure
            return False


    def decision_tree(self, train_df, min_samples, max_depth, counter=0):

        '''only one arg needed (df). fitting this training df will account for
        splitting data into X and y'''

        # if this is our first potential split
        if counter == 0:

            # set this global variable so we can use it each time we call decision_tree()
            global feature_names
            feature_names = train_df.columns

            # get our df values
            data = train_df.values

        # if we have recursively reached this point
        else:

            # our 'train_df' is actually already an array upon recursion
            data = train_df
            
        # base case: if our impurity for a subtree is 0 or we have reached our 
        # maximum depth or min samples
        if (self.purity(data)) or (len(data) < min_samples) or (counter == max_depth):
            
            # at this point we'll have to make a judgment call and classify it based on the majority
            return self.make_classification(data)

        # if we haven't reach one of our stopping points
        else:

            # increment counter
            counter += 1

            # get a dictionary of our potential splits
            potential_splits = self.potential_splits(data)
            
            # find the best split
            split_feature, split_threshold = self.find_best_split(data, potential_splits)

            # get the data below and above
            data_below_threshold, data_above_threshold = self.split_data(data, split_feature, split_threshold)

            # store feature name as string
            feature_name_as_string = feature_names[split_feature]

            # feature_name <= threshold value
            split_question = f"{feature_name_as_string} <= {split_threshold}"

            # create a dictionary for these split_questions 
            subtree = {split_question: []}

            # recursion on our true values
            answer_true = self.decision_tree(data_below_threshold, min_samples, max_depth, counter)

            # recursion on our false values
            answer_false = self.decision_tree(data_above_threshold, min_samples, max_depth, counter)

            # if both answers are the same class
            if answer_true == answer_false:

                # choose one to be the subtree
                subtree = answer_true
            
            # if answers result in different class
            else:

                # append to dictionary
                subtree[split_question].append(answer_true)
                subtree[split_question].append(answer_false)
            
            return subtree

    def classify_observation(self, observation, tree):

        # store the current question 
        split_question = list(tree.keys())[0]

        # grab the feature name and value 
        feature_name, _, value = split_question.split()

        # if the row at that feature column is less than the threshold
        if observation[feature_name] <= float(value):

            # answer yes, it's under the threshold
            answer = tree[split_question][0]

        # if the row at that feature column has exceeded the threshold
        else:

            # answer no, it has exceeded the threshold
            answer = tree[split_question][1]
        

        # if the answer is not a dictionary
        if not isinstance(answer, dict):

            # return answer as it is a class label
            return answer

        # if the answer is a dictionary
        else:

            # recursion with the 'answer' subtree as the tree argument
            return self.classify_observation(observation, answer)
        
    def predict(self, test_df):

        # if a tree has been fitted
        if self.tree:
            
            # store true labels for our test_df
            y_test_true = test_df.iloc[:, -1]
            
            # create a new column for our predictions
            test_df['predictions'] = test_df.apply(self.classify_observation, axis=1, args=(self.tree,))
            
            # return True or False where our predictions are equal to our labels
            test_df['correct_predictions'] = test_df['predictions'] == y_test_true

            # return how accurate the predictions are
            return ("Accuracy:", test_df['correct_predictions'].mean())
        
        # if a tree wasn't fitted
        else:
            print("NotFittedError: Please fit your decision tree first!")

In [83]:
data = train_df.values
data[:5]

array([[1, 5.1, 3.5, 1.4, 0.2, 'Iris-setosa'],
       [2, 4.9, 3.0, 1.4, 0.2, 'Iris-setosa'],
       [3, 4.7, 3.2, 1.3, 0.2, 'Iris-setosa'],
       [4, 4.6, 3.1, 1.5, 0.2, 'Iris-setosa'],
       [5, 5.0, 3.6, 1.4, 0.2, 'Iris-setosa']], dtype=object)

In [531]:
overall_entropy(train_df.values)

[0.31707317 0.34146341 0.34146341]


1.5840970252657152

In [539]:
potential_splits = find_potential_splits(train_df.values)

In [540]:
find_best_split(train_df.values, potential_splits)

0.01287312049569506
0 4.35
0.03856807071294372
0.05137767157946454
0.10219673763065067
0.1272351322978637
0.1761418903558301
0.32803556179607246
0.4847610605285572
0.5811252011724818
0.6306854382727127
0.642113942702162
0.7594692874469247
0.8919877111004573
1.0396058320300883
1.1447485765438545
1.2169038848912836
1.3077703605240751
1.3698286921579919
1.4508397229358405
1.6176204413295296
1.780373089309827
1.905168724426641
1.878078041889502
1.9313455927223449
1.8765985183052571
1.83623654537148
1.447741499679517
1.462410060029398
1.5049378361544672
1.5186511138543943
1.5321467482150686
1.5714008512276023
0.012880334846127888
0.038635806228670515
0.12370393348864778
0.1664243050738268
0.36803281538650323
0.475869376546437
0.6239572042405866
0.8697110773571067
0.9829601856912552
1.4563183038711567
1.6760184733352728
1.8117407553626144
1.8008257510479153
1.83445289988662
2.0506017750570713
2.0526413222891575
2.1684091041021207
1.5427228285177488
1.5567318184434056
1.570520700474797
0.0128

(0, 4.35)

In [538]:
    def entropy(data):

        # labels for input data
        labels = data[:,-1]

        # get unique labels along with their counts
        _, label_counts = np.unique(labels, return_counts=True)

        # array of probabilities of each label
        probabilities = label_counts / label_counts.sum()
                
        # return entropy
        return sum(probabilities * -np.log2(probabilities))

    def make_classification(data):

        '''Once the max depth or min samples or purity is 1, 
        we classify the data with whatever the majority of the labels are'''

        # labels for input data
        labels = data[:,-1]

        # instantiate a Counter object on the labels
        counter = Counter(labels)

        # return the most common class/label
        return counter.most_common(1)[0][0]

    def split_data(data, split_feature, split_threshold):

        # array of only the split_feature
        feature_values = data[:,split_feature]

        # array where feature values do not exceed threshold, array where does exceed threshold
        return data[feature_values <= split_threshold], data[feature_values > split_threshold]

    def overall_entropy(data_below_threshold, data_above_threshold):

        '''Overall entropy'''

        p = len(data_below_threshold) / (len(data_below_threshold) + len(data_above_threshold))

        return p * entropy(data_below_threshold) + p * entropy(data_above_threshold)

    def find_potential_splits(data):
        
        # dictionary of splits
        potential_splits = {}

        # store the number of features (not including labels/target)
        n_features = len(data[0])

        # for each feature in possible features
        for feature in range(n_features - 1):

            # for our dictionary, each feature should be a key
            potential_splits[feature] = []

            # we need to iterate through each feature's unique values
            unique_values_for_feature = np.unique(data[:, feature])

            for index in range(len(unique_values_for_feature)):
                
                if index != 0:

                    # we need to partition the data, we need the midpoint between the unique values
                    current = unique_values_for_feature[index]
                    prev = unique_values_for_feature[index - 1]
                    midpoint = (current + prev) / 2

                    # for our dictionary each value should be a midpoint between the 
                    # unique values for that feature
                    potential_splits[feature].append(midpoint)

        # return dictionary
        return potential_splits

    def find_best_split(data, potential_splits):

        lowest_entropy = 9999

        # for each dictionary key
        for key in potential_splits:
            
            # for each value for that key
            for value in potential_splits[key]:

                # split our data into on that threshold (value)
                data_below_threshold, data_above_threshold = split_data(
                    data=data, 
                    split_feature=key,
                    split_threshold=value)
                
                # calculate entropy at this split
                entropy_for_this_split = overall_entropy(data_below_threshold, data_above_threshold)
                
                print(entropy_for_this_split)

                # if entropy at this split is lower than the lowest entropy found so far
                if entropy_for_this_split < lowest_entropy:

                    # the entropy at this split is now the lowest 
                    lowest_entropy = entropy_for_this_split

                    # keep a record of this key, value pair
                    best_split_feature = key
                    best_split_threshold = value
                    
                    print(best_split_feature, best_split_threshold)

        # return the best potential split
        return best_split_feature, best_split_threshold
    

    def purity(data):

        # last column of the df must be the labels!
        labels = data[:, -1]

        # if data has only one kind of label
        if len(np.unique(labels)) == 1:

            # it is pure
            return True

        # if data has a few different labels still
        else:

            # it isn't pure
            return False


    def decision_tree(train_df, min_samples, max_depth, counter=0):

        '''only one arg needed (df). fitting this training df will account for
        splitting data into X and y'''

        # if this is our first potential split
        if counter == 0:

            # set this global variable so we can use it each time we call decision_tree()
            global feature_names
            feature_names = train_df.columns

            # get our df values
            data = train_df.values

        # if we have recursively reached this point
        else:

            # our 'train_df' is actually already an array upon recursion
            data = train_df
            
        # base case: if our impurity for a subtree is 0 or we have reached our 
        # maximum depth or min samples
        if (purity(data)) or (len(data) < min_samples) or (counter == max_depth):
            
            # at this point we'll have to make a judgment call and classify it based on the majority
            return make_classification(data)
        

        # if we haven't reach one of our stopping points
        else:

            # increment counter
            counter += 1

            # get a dictionary of our potential splits
            potential_splits = find_potential_splits(data)
            
            
            # find the best split
            split_feature, split_threshold = find_best_split(data, potential_splits)
            
            print('Best Split:', split_feature, split_threshold)

            # get the data below and above
            data_below_threshold, data_above_threshold = split_data(data, split_feature, split_threshold)

            # store feature name as string
            feature_name_as_string = feature_names[split_feature]

            # feature_name <= threshold value
            split_question = f"{feature_name_as_string} <= {split_threshold}"

            # create a dictionary for these split_questions 
            subtree = {split_question: []}

            # recursion on our true values
            answer_true = decision_tree(data_below_threshold, min_samples, max_depth, counter)

            # recursion on our false values
            answer_false = decision_tree(data_above_threshold, min_samples, max_depth, counter)

            # if both answers are the same class
            if answer_true == answer_false:

                # choose one to be the subtree
                subtree = answer_true
            
            # if answers result in different class
            else:

                # append to dictionary
                subtree[split_question].append(answer_true)
                subtree[split_question].append(answer_false)
            
            return subtree

        
    def predict(test_df):

            
        # store true labels for our test_df
        y_test_true = test_df.iloc[:, -1]

        # create a new column for our predictions
        test_df['predictions'] = test_df.apply(self.classify_observation, axis=1, args=(self.tree,))

        # return True or False where our predictions are equal to our labels
        test_df['correct_predictions'] = test_df['predictions'] == y_test_true

        # return how accurate the predictions are
        return ("Accuracy:", test_df['correct_predictions'].mean())

In [527]:
    def classify_observation(observation, tree):

        # store the current question 
        split_question = list(tree.keys())[0]
        
        print(tree.keys())

        # grab the feature name and value 
        feature_name, _, value = split_question.split()

#         # if the row at that feature column is less than the threshold
#         if observation[feature_name] <= float(value):

#             # answer yes, it's under the threshold
#             answer = tree[split_question][0]

#         # if the row at that feature column has exceeded the threshold
#         else:

#             # answer no, it has exceeded the threshold
#             answer = tree[split_question][1]

#         # if the answer is not a dictionary
#         if not isinstance(answer, dict):

#             # return answer as it is a class label
#             return answer

#         # if the answer is a dictionary
#         else:

#             # recursion with the 'answer' subtree as the tree argument
#             return classify_observation(observation, answer)

In [528]:
tree = decision_tree(train_df, min_samples=2, max_depth=5, counter=0)

0 4.35
Best Split: 0 4.35
0 4.45
1 2.1
2 1.1
Best Split: 2 1.1
0 4.45
1 2.1
2 1.25
Best Split: 2 1.25
0 4.45
1 2.1
Best Split: 1 2.1
0 4.45
Best Split: 0 4.45


In [515]:
tree

{'SepalLengthCm <= 4.35': ['Iris-setosa', 'Iris-versicolor']}

In [516]:
df.tail()

,SepalLengthCm,SepalWidthCm,PetalLengthCm,PetalWidthCm,Species
145,6.7,3.0,5.2,2.3,Iris-virginica
146,6.3,2.5,5.0,1.9,Iris-virginica
147,6.5,3.0,5.2,2.0,Iris-virginica
148,6.2,3.4,5.4,2.3,Iris-virginica
149,5.9,3.0,5.1,1.8,Iris-virginica


In [521]:
obs = test_df.iloc[10]
print(obs)

SepalLengthCm                      5.4
SepalWidthCm                         3
PetalLengthCm                      4.5
PetalWidthCm                       1.5
Species                Iris-versicolor
predictions                Iris-setosa
correct_predictions              False
Name: 84, dtype: object


In [522]:
classify_observation(obs, tree)

dict_keys(['SepalLengthCm <= 4.35'])
